# SUMÁRIO

- [1. INFOS API DATAJUD](#1-infos-api-datajud)

- [EXEMPLOS DE CONSULTA INDIVIDUAL](#testes-consulta-individual)

### 1. INFOS API DATAJUD

#### ENDPOINT

- https://api-publica.datajud.cnj.jus.br/api_publica_tjsp/_search

#### TESTES CONSULTA INDIVIDUAL

[VOLTAR AO SUMÁRIO](#sumário)

### 1. CARREGAR PACOTES 

In [4]:
import requests
import json
import ast

import pandas as pd
pd.set_option('display.max_rows', 100) 
import numpy as np

import concurrent.futures
from time import sleep
from tqdm import tqdm

In [5]:
API_KEY = "cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw=="
url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjsp/_search"
headers = {
    'Authorization': f'ApiKey {API_KEY}',
    'Content-Type': 'application/json'
}

def consultar_processo(numero):
    payload = json.dumps({
        "query": {
            "match": {
                "numeroProcesso": numero
            }
        }
    })

    try:
        response = requests.post(url, headers=headers, data=payload, timeout=30)
        response.raise_for_status()
        data = response.json()

        resultados = []
        for hit in data.get("hits", {}).get("hits", []):
            registro = hit["_source"]
            #registro["_id"] = hit.get("_id")
            resultados.append(registro)

        return resultados

    except Exception as e:
        print(f"❌ Erro no processo {numero}: {e}")
        return []

def processar_em_lotes(lista, max_threads=5):
    todos_resultados = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_threads) as executor:
        futures = {executor.submit(consultar_processo, numero): numero for numero in lista}

        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
            resultado = future.result()
            todos_resultados.extend(resultado)
            sleep(0.1)  

    return todos_resultados


In [ ]:
lote001 = pd.read_parquet('./app10/resultado-api/lotes/lote_001.parquet')
# lote1200 = pd.read_parquet('./app10/resultado-api/lotes/lote_1200.parquet')

In [ ]:
lote001['movimentos'][0]

In [ ]:
consultar_processo('20501854119928260405')

In [ ]:
df = pd.DataFrame(consultar_processo('00014874419948260445'))

df.movimentos[0]

In [ ]:
df['movimentos'][0]

#### 2. Função para expandir colunas com dicionários

In [ ]:
def expandir_colunas(df, colunas_alvo, chaves=['codigo', 'nome'], como_string=True, separador='|'):
    """
    Expande colunas contendo listas de dicionários (ou dicts), criando novas colunas para cada chave.
    Os valores podem ser retornados como listas ou strings separadas por vírgula (default).
    Mantém a ordem original das colunas, inserindo as novas logo após a original.
    """
    df_exp = df.copy()
    novas_colunas = {}

    def extrair_primeiros(lista_ou_dict, chave):
        if isinstance(lista_ou_dict, dict):
            val = lista_ou_dict.get(chave, None)
            return str(val) if como_string else val
        elif isinstance(lista_ou_dict, list):
            try:
                valores = [str(d.get(chave)) for d in lista_ou_dict if isinstance(d, dict)]
                return separador.join(valores) if como_string else valores
            except:
                return None
        return None

    for col in colunas_alvo:
        if col not in df_exp.columns:
            continue
        valores_expandidos = df_exp[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
        for chave in chaves:
            nova_coluna = f"{col}_{chave}"
            # Garante que só cria a coluna uma vez
            if nova_coluna not in novas_colunas:
                novas_colunas[nova_coluna] = valores_expandidos.apply(lambda v: extrair_primeiros(v, chave))

    # Insere as novas colunas logo após cada coluna original
    for col in colunas_alvo:
        if col not in df_exp.columns:
            continue
        idx_col = df_exp.columns.get_loc(col)
        insert_at = idx_col + 1
        for chave in chaves:
            nova_coluna = f"{col}_{chave}"
            if nova_coluna in df_exp.columns:
                continue  # Não sobrescreve se já existe
            df_exp.insert(insert_at, nova_coluna, novas_colunas[nova_coluna])
            insert_at += 1

    if 'numeroProcesso' in df_exp.columns:
        cols = df_exp.columns.tolist()
        cols.remove('numeroProcesso')
        df_exp = df_exp[['numeroProcesso'] + cols]


    return df_exp

In [ ]:
lote001 = pd.read_parquet('./app10/resultado-api/lotes/lote_001.parquet')

lote001.head()

In [ ]:
[lote001.assuntos[i] for i in range(len(lote001))]

In [ ]:
colunas = ['classe', 'sistema', 'formato', 'orgaoJulgador', 'assuntos']
df_expandido = expandir_colunas(lote001, colunas)

df_expandido.head()

In [ ]:
df_expandido.columns

In [ ]:
df_expandido[df_expandido['numeroProcesso'] == '10018662020248260411'][['assuntos','assuntos_codigo','assuntos_nome']]

In [ ]:
lote001.head()

In [ ]:
lote001['movimentos'][0]

In [ ]:
def extrair_movimentos_completos (df: pd.DataFrame) -> pd.DataFrame:
    todos_movimentos = []

    for _, row in df.iterrows():
        processo_id = row.get("_id")
        numero_processo = row.get("numeroProcesso")
        movimentos = row.get("movimentos")
        data_download = row.get("data_download")

        if not isinstance(movimentos, (list, np.ndarray)):
            continue

        for mov in movimentos:
            movimento_info = {
                "_id": processo_id,
                "numeroProcesso": numero_processo,
                "movimentos_codigo": mov.get("codigo"),
                "movimentos_nome": mov.get("nome"),
                "movimentos_dataHora": mov.get("dataHora"),
                "movimentos_orgaoJulgador": mov.get("orgaoJulgador"),
                "movimentos_orgaoJulgador_codigoOrgao": None,
                "movimentos_orgaoJulgador_nomeOrgao": None,
                "movimentos_complementosTabelados": None,
                "movimentos_complementosTabelados_codigo": None,
                "movimentos_complementosTabelados_descricao": None,
                "movimentos_complementosTabelados_valor": None,
                "movimentos_complementosTabelados_nome": None,
                "data_download": data_download
            }

            # Orgao julgador (dict)
            orgao = mov.get("orgaoJulgador")
            if isinstance(orgao, dict):
                movimento_info["movimentos_orgaoJulgador_codigoOrgao"] = orgao.get("codigoOrgao")
                movimento_info["movimentos_orgaoJulgador_nomeOrgao"] = orgao.get("nomeOrgao")

            # Complementos Tabelados (list ou ndarray de dicts)
            complementos = mov.get("complementosTabelados")

            if isinstance(complementos, np.ndarray):
                complementos = complementos.tolist()

            if isinstance(complementos, list) and len(complementos) > 0:
                for comp in complementos:
                    comp_info = movimento_info.copy()
                    comp_info["movimentos_complementosTabelados"] = complementos
                    comp_info["movimentos_complementosTabelados_codigo"] = comp.get("codigo")
                    comp_info["movimentos_complementosTabelados_descricao"] = comp.get("descricao")
                    comp_info["movimentos_complementosTabelados_valor"] = comp.get("valor")
                    comp_info["movimentos_complementosTabelados_nome"] = comp.get("nome")
                    todos_movimentos.append(comp_info)
            else:
                # Sem complementos, mas ainda queremos registrar o movimento
                todos_movimentos.append(movimento_info)

    df_movimentos = pd.DataFrame(todos_movimentos)

    if not df_movimentos.empty and "movimentos_dataHora" in df_movimentos.columns:
        df_movimentos["movimentos_dataHora"] = pd.to_datetime(df_movimentos["movimentos_dataHora"], errors="coerce")
        ordem_id_original = pd.CategoricalDtype(categories = df['_id'].to_list(),ordered=True)
        df_movimentos['_id'] = df_movimentos['_id'].astype(ordem_id_original)

        df_movimentos = df_movimentos.sort_values(by=['_id', "movimentos_dataHora"], ascending=[True,False])
        
        df_movimentos = df_movimentos.reset_index(drop=True)
    
    return df_movimentos


In [ ]:
lote001.iloc[[0]]

In [ ]:
lote001.iloc[[0]].movimentos[0]

In [1]:
def extrair_movimentos_completos (df: pd.DataFrame) -> pd.DataFrame:
    todos_movimentos = []

    for _, row in df.iterrows():
        processo_id = row.get("_id")
        numero_processo = row.get("numeroProcesso")
        movimentos = row.get("movimentos")
        data_download = row.get("data_download")

        if not isinstance(movimentos, (list, np.ndarray)):
            continue

        for mov in movimentos:
            movimento_info = {
                "_id": processo_id,
                "numeroProcesso": numero_processo,
                "movimentos_codigo": mov.get("codigo"),
                "movimentos_nome": mov.get("nome"),
                "movimentos_dataHora": mov.get("dataHora"),
                "movimentos_orgaoJulgador": mov.get("orgaoJulgador"),
                "movimentos_orgaoJulgador_codigoOrgao": None,
                "movimentos_orgaoJulgador_nomeOrgao": None,
                "movimentos_complementosTabelados": None,
                "movimentos_complementosTabelados_codigo": None,
                "movimentos_complementosTabelados_descricao": None,
                "movimentos_complementosTabelados_valor": None,
                "movimentos_complementosTabelados_nome": None,
                "data_download": data_download
            }

            # Orgao julgador (dict)
            orgao = mov.get("orgaoJulgador")
            if isinstance(orgao, dict):
                movimento_info["movimentos_orgaoJulgador_codigoOrgao"] = orgao.get("codigoOrgao")
                movimento_info["movimentos_orgaoJulgador_nomeOrgao"] = orgao.get("nomeOrgao")

            # Complementos Tabelados (list ou ndarray de dicts)
            complementos = mov.get("complementosTabelados")

            if isinstance(complementos, np.ndarray):
                complementos = complementos.tolist()

            if isinstance(complementos, list) and len(complementos) > 0:
                for comp in complementos:
                    comp_info = movimento_info.copy()
                    comp_info["movimentos_complementosTabelados"] = complementos
                    comp_info["movimentos_complementosTabelados_codigo"] = comp.get("codigo")
                    comp_info["movimentos_complementosTabelados_descricao"] = comp.get("descricao")
                    comp_info["movimentos_complementosTabelados_valor"] = comp.get("valor")
                    comp_info["movimentos_complementosTabelados_nome"] = comp.get("nome")
                    todos_movimentos.append(comp_info)
            else:
                # Sem complementos, mas ainda queremos registrar o movimento
                todos_movimentos.append(movimento_info)

    df_movimentos = pd.DataFrame(todos_movimentos)

    if not df_movimentos.empty and "movimentos_dataHora" in df_movimentos.columns:
        df_movimentos["movimentos_dataHora"] = pd.to_datetime(df_movimentos["movimentos_dataHora"], errors="coerce")
        ordem_id_original = pd.CategoricalDtype(categories = df['_id'].to_list(),ordered=True)
        df_movimentos['_id'] = df_movimentos['_id'].astype(ordem_id_original)

        df_movimentos = df_movimentos.sort_values(by=['_id', "movimentos_dataHora"], ascending=[True,False])
        
        df_movimentos = df_movimentos.reset_index(drop=True)
    
    return df_movimentos

NameError: name 'pd' is not defined

In [ ]:
df_movs = extrair_movimentos_completos(df)
df_movs[['_id','numeroProcesso','movimentos_dataHora','movimentos_nome']].sort_values(by=['movimentos_dataHora'], ascending=True).head(5)


In [ ]:
df_movs[['_id','numeroProcesso','movimentos_dataHora','movimentos_nome']].head(5)

In [ ]:
df['movimentos'][0]

In [ ]:
df_movs.groupby('_id').size().reset_index(name='quantidade_movimentos').sort_values(by='quantidade_movimentos', ascending=False)


In [ ]:
lote001['movimentos'][0]

In [ ]:
df_movs[df_movs['_id'] == 'TJSP_JE_15019982920248260408'].info()

In [ ]:
lote001[lote001['_id'] == 'TJSP_JE_15019982920248260408'][['movimentos']]

In [ ]:
df_movs[df_movs['_id'] == 'TJSP_JE_15019982920248260408']

### DEPURANDO FUNÇÃO PARA EXTRAÇÃO DOS MOVIMENTOS

In [ ]:
df_movs.shape

#### 1. Verificar colunas esperadas

In [ ]:
colunas_esperadas = [
    "_id",
    "movimentos_codigo",
    "movimentos_nome",
    "movimentos_dataHora",
    "movimentos_complementosTabelados",
    "movimentos_complementosTabelados_codigo",
    "movimentos_complementosTabelados_descricao",
    "movimentos_complementosTabelados_valor",
    "movimentos_complementosTabelados_nome",
    "movimentos_orgaoJulgador",
    "movimentos_orgaoJulgador_codigoOrgao",
    "movimentos_orgaoJulgador_nomeOrgao",
]

colunas_faltantes = set(colunas_esperadas) - set(df_movs.columns)

print("Colunas faltantes:", colunas_faltantes if colunas_faltantes else "Nenhuma")



#### 2. Verificar tipos de dados


In [ ]:
print(df_movs.dtypes)


#### 3. Inspecionar valores não nulos por coluna

- Checar se algum campo está completamente vazio

In [ ]:
print(df_movs.notnull().sum())


#### 4. Exibir exemplos onde complementos foram extraídos

In [ ]:
df_movs[df_movs["movimentos_complementosTabelados"].notnull()]


#### 5. Verificar valores únicos para campos categóricos

In [ ]:
print(df_movs["movimentos_nome"].unique())
print(df_movs["movimentos_complementosTabelados_nome"].unique())


#### 6. Verificar casos com dados aninhados incompletos ou malformados

In [ ]:
df_movs[
    df_movs["movimentos_complementosTabelados"].notnull() &
    df_movs["movimentos_complementosTabelados_codigo"].isnull()
]


####  7. Contar registros totais e com complementos

In [ ]:
print("Total de movimentos extraídos:", len(df_movs))
print("Com complementos:", df_movs["movimentos_complementosTabelados"].notnull().sum())


#### 8. Verificar se há valores inconsistentes

In [ ]:
df_movs[
    df_movs["movimentos_complementosTabelados"].notnull() &
    df_movs["movimentos_complementosTabelados_codigo"].isnull() &
    df_movs["movimentos_complementosTabelados_nome"].isnull()
]


### Variação extração movimentos 

Extração dos campos

- movimentos.orgaoJulgador	
- movimentos.orgaoJulgador.codigoOrgao
- movimentos.orgaoJulgador.nomeOrgao	

In [ ]:
def extrair_movimentos2(df: pd.DataFrame) -> pd.DataFrame:
    """Extrai campos dos movimentos processuais, incluindo orgaoJulgador e complementosTabelados."""

    movimentos = df[["_id", "movimentos"]].explode("movimentos").reset_index(drop=True)

    movimentos["movimentos_codigo"] = movimentos["movimentos"].apply(lambda x: x.get("codigo") if isinstance(x, dict) else None)
    movimentos["movimentos_nome"] = movimentos["movimentos"].apply(lambda x: x.get("nome") if isinstance(x, dict) else None)
    movimentos["movimentos_dataHora"] = movimentos["movimentos"].apply(lambda x: x.get("dataHora") if isinstance(x, dict) else None)

    # ✅ Extraindo o dicionário bruto do orgaoJulgador
    movimentos["movimentos_orgaoJulgador"] = movimentos["movimentos"].apply(
        lambda x: x.get("orgaoJulgador") if isinstance(x, dict) else None
    )

    # 🔍 Extraindo os subcampos
    movimentos["movimentos_orgaoJulgador_codigoOrgao"] = movimentos["movimentos_orgaoJulgador"].apply(
        lambda x: x.get("codigoOrgao") if isinstance(x, dict) else None
    )
    movimentos["movimentos_orgaoJulgador_nomeOrgao"] = movimentos["movimentos_orgaoJulgador"].apply(
        lambda x: x.get("nomeOrgao") if isinstance(x, dict) else None
    )

    # ✅ ComplementosTabelados
    movimentos["movimentos_complementosTabelados"] = movimentos["movimentos"].apply(
        lambda x: x.get("complementosTabelados") if isinstance(x, dict) else None
    )

    movimentos = movimentos.explode("movimentos_complementosTabelados").reset_index(drop=True)

    movimentos["movimentos_complementosTabelados_codigo"] = movimentos["movimentos_complementosTabelados"].apply(
        lambda x: x.get("codigo") if isinstance(x, dict) else None
    )
    movimentos["movimentos_complementosTabelados_descricao"] = movimentos["movimentos_complementosTabelados"].apply(
        lambda x: x.get("descricao") if isinstance(x, dict) else None
    )
    movimentos["movimentos_complementosTabelados_valor"] = movimentos["movimentos_complementosTabelados"].apply(
        lambda x: x.get("valor") if isinstance(x, dict) else None
    )
    movimentos["movimentos_complementosTabelados_nome"] = movimentos["movimentos_complementosTabelados"].apply(
        lambda x: x.get("nome") if isinstance(x, dict) else None
    )

    # 🎯 Selecionando apenas as colunas desejadas
    colunas_finais = [
        "_id",
        "movimentos_codigo",
        "movimentos_nome",
        "movimentos_dataHora",
        "movimentos_orgaoJulgador",
        "movimentos_orgaoJulgador_codigoOrgao",
        "movimentos_orgaoJulgador_nomeOrgao",
        "movimentos_complementosTabelados",
        "movimentos_complementosTabelados_codigo",
        "movimentos_complementosTabelados_descricao",
        "movimentos_complementosTabelados_valor",
        "movimentos_complementosTabelados_nome"
    ]

    return movimentos[colunas_finais]


In [ ]:
df = extrair_movimentos2(lote001)

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
print(df.notnull().sum())

In [ ]:
# Verifica se ao menos um movimento possui 'orgaoJulgador'
existe_orgaoJulgador = lote001['movimentos'].explode().dropna().apply(
    lambda x: isinstance(x, dict) and 'orgaoJulgador' in x
).sum()

print(f"Número de movimentos com 'orgaoJulgador': {existe_orgaoJulgador}")
